<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myXH_Return_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import yfinance as yf
import pandas as pd

def calculate_stock_gains(start_date, end_date, stock_list):
    """
    Calculate percentage gains for a list of NSE stocks over a specified date range.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        stock_list (list): List of NSE stock symbols (e.g., ['RELIANCE.NS', 'TCS.NS']).

    Returns:
        pd.DataFrame: DataFrame containing stock symbols and their percentage gains.
    """
    data = []

    for stock in stock_list:
        try:
            # Fetch stock data for the given date range
            stock_data = yf.Ticker(stock + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            stock_data = stock_data[(stock_data.index <= end_date) & (stock_data.index >= start_date)]

            if not stock_data.empty:
                # Calculate percentage gain
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percentage_gain = ((end_price - start_price) / start_price) * 100

                data.append({
                    'Symbol': stock,
                    'Start Price': start_price,
                    'End Price': end_price,
                    'Gains (%)': round(percentage_gain, 2)
                })
            else:
                data.append({
                    'Symbol': stock,
                    'Start Price': None,
                    'End Price': None,
                    'Gains (%)': None
                })
        except Exception as e:
            data.append({
                'Symbol': stock,
                'Start Price': None,
                'End Price': None,
                'Gains (%)': None,
                'Error': str(e)
            })

    # Create and return DataFrame
    return pd.DataFrame(data)

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# Example usage:
if __name__ == "__main__":
    start_date = "2023-06-05"
    end_date = "2024-09-27"


    pps_df = get_mypps_df()
    scr_df = get_myscreen_df()

    df = pd.merge(pps_df, scr_df, on = 'Symbol')

    #stock_list = df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values

    list_x40 = list(df[df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
    list_x5k = list(df[df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
    list_x40n = list(df[df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
    list_hc = list(df[df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
    list_ox40 = list(df[df['MBQ'].str.contains('OX40', na=False)]['Symbol'].values)
    list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)
    #len(set(list_x40 + list_x5k) - set( list_x40n))
    stock_list = list(set(list_x40 + list_x5k + list_hc)- set(list_ox40 + list_ox40n))

    stock_list = df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values

    #stock_list = ["RELIANCE", "TCS", "INFY", "HDFCBANK"]

    result_df = calculate_stock_gains(start_date, end_date, stock_list)
    result_df = pd.merge(result_df, df, on = 'Symbol')
    result_df['Conviction'] = result_df['Conviction'] + '-'+ result_df['CapType']
    cols = ['Symbol', 'Conviction', 'MBQ', 'InFolio', 'Start Price', 'End Price', 'Gains (%)']
    result_df = result_df[cols]
    print(str(round(result_df['Gains (%)'].mean())) + '%')

result_df.sort_values(by = 'Gains (%)', ascending=False)

69%


,Symbol,Conviction,MBQ,InFolio,Start Price,End Price,Gains (%)
10,BSE,X-LC,X40N,NaN,187.849701,1211.495605,544.93
3,BAJAJ-AUTO,X-LC,X40/X5K,NaN,4421.336914,12353.318359,179.40
21,HDFCAMC,X-LC,X40,NaN,923.876404,2169.560059,134.83
37,M&M,X-LC,X5K/X40N,NaN,1359.964844,3158.276123,132.23
24,HEROMOTOCO,H-LC,AR,NaN,2649.979736,5724.968262,116.04
7,BHARTIARTL,H-LC,X5K,NaN,819.231995,1720.219116,109.98
45,SIEMENS,H-LC,AR,1.0,2052.004395,4267.249023,107.96
8,BOSCHLTD,X-LC,X40N,NaN,18127.078125,37467.566406,106.69
0,ADANIPORTS,X-LC,XY24/X5K/X40N,NaN,727.193359,1449.647095,99.35
51,TORNTPHARM,H-LC,NaN,NaN,1732.476318,3449.084229,99.08


In [7]:
result_df.sort_values(by = 'Gains (%)', ascending=False).tail(20)

,Symbol,Conviction,MBQ,InFolio,Start Price,End Price,Gains (%)
53,VBL,X-LC,X40N,1.0,606.818970,475.942261,-21.57
36,LTIM,H-LC,X200,NaN,6037.020996,4624.844727,-23.39
42,RELIANCE,X-LC,XY25/X40/X5K,1.0,1520.103882,1157.277954,-23.87
43,SBILIFE,H-LC,AR,NaN,1879.073608,1390.453613,-26.00
26,HINDUNILVR,X-LC,XY25/X5K/X40,1.0,2878.987305,2118.247559,-26.42
9,BRITANNIA,H-LC,XY25/AR,NaN,6187.709473,4516.017578,-27.02
0,ADANIPORTS,X-LC,XY24/X5K/X40N,NaN,1449.647095,1052.827759,-27.37
18,GODREJCP,X-LC,X40N,NaN,1360.487305,978.016296,-28.11
34,LICI,H-LC,XY25/X5K,NaN,1010.846497,723.605225,-28.42
14,DLF,H-LC,AR,NaN,907.412842,638.628784,-29.62
